In [1]:
# %%capture
# %pip install roiextractors
# %pip install neuroconv==0.2.4
# %pip install tifffile
# %pip install opencv-python-headless

In [27]:
dandiset_id = None
if dandiset_id is None:
    print('None')
else:
    print(dandiset_id)
    

None


In [14]:
import os
from concurrent.futures import as_completed, ProcessPoolExecutor
from pathlib import Path
from warnings import warn

# from neuroconv.tools.data_transfers import automatic_dandi_upload
from nwbinspector import inspect_nwb
from nwbinspector.inspector_tools import format_messages, save_report
from tqdm import tqdm
import datajoint as dj
import pandas as pd

# dj.config["database.host"] = "db.datajoint.com"
# dj.config["database.user"] = "microns"
# dj.config["database.password"] = "microns2023"

# nda = dj.create_virtual_module('nda', 'microns_phase3_nda')
# from phase3 import nda
from microns_phase3 import nda


from tools.intervals import add_trials
from tools.nwb_helpers import start_nwb
from tools.ophys import add_ophys
# from tools.times import get_stimulus_times, get_frame_times, get_trial_times
from tools.times import resample_flips

from micronsnwbconverter import MICrONSNWBConverter
from tools.behavior import find_earliest_timestamp, add_eye_tracking, add_treadmill

In [3]:
# Source data file paths
scan_keys = list(nda.Scan.proj())

# The list of file paths to the imaging data
ophys_file_version = 2
ophys_file_directory = f"/mnt/at-export01/17797-export/two_photon_functional_scans/v{ophys_file_version}/"
ophys_file_paths = [Path(ophys_file_directory + '_'.join(['functional','scan','17797',
                                                         *[str(s) for s in scan_key.values()],
                                                         'v'+str(ophys_file_version)+'.tif'])) 
                    for scan_key in scan_keys]

# The list of file paths to the stimulus movie files
movie_file_version = 4
movie_file_directory = f"/mnt/at-export01/17797-export/stimulus_movies/v{movie_file_version}/"
movie_file_paths = [Path(movie_file_directory + '_'.join(['stimulus','17797',
                                                         *[str(s) for s in scan_key.values()],
                                                         'v'+str(movie_file_version)+'.avi'])) 
                    for scan_key in scan_keys]

# The file path to the NWB files
nwb_output_path = Path("/mnt/at-export01/17797-export/DANDI_NWB/")
nwbfile_list = [nwb_output_path / ophys_file_path.stem / f"{ophys_file_path.stem}.nwb" for ophys_file_path in ophys_file_paths]

# create directories for NWB files
nwbfile_folder_paths = [nwb_output_path / ophys_file_path.stem for ophys_file_path in ophys_file_paths]
_ = [os.makedirs(nwbfile_folder_path, exist_ok=True) for nwbfile_folder_path in nwbfile_folder_paths]

In [4]:
ophys_file_path = ophys_file_paths[0]
nwbfile_path = nwbfile_list[0]
stimulus_movie_file_path = movie_file_paths[0]
verbose = True

In [5]:
scan_key = dict(
    session=Path(ophys_file_path).stem.split("_")[3],
    scan_idx=Path(ophys_file_path).stem.split("_")[4],
)

source_data = dict(
    Ophys=dict(file_path=str(ophys_file_path), scan_key=scan_key),
    Video=dict(file_paths=[str(stimulus_movie_file_path)]),
)

movie_times,_ = resample_flips(scan_key)
frame_times = (nda.ScanTimes & scan_key).fetch1('frame_times')
trial_times = pd.DataFrame((nda.Trial & scan_key).fetch())

# Shifting times to earliest provided behavioral timestamp when necessary
pupil_timestamps = (nda.RawManualPupil & scan_key).fetch1("pupil_times")
treadmill_timestamps = (nda.RawTreadmill & scan_key).fetch1("treadmill_timestamps")

earliest_timestamp_in_behavior = find_earliest_timestamp(
    behavior_timestamps_arrays=[pupil_timestamps, treadmill_timestamps],
)

if earliest_timestamp_in_behavior < 0:
    warn(
        "Writing behavior data to NWB with negative timestamps is not recommended,"
        f"times are shifted to the earliest behavioral timestamp by {abs(earliest_timestamp_in_behavior)} seconds."
    )
    pupil_timestamps = pupil_timestamps + abs(earliest_timestamp_in_behavior)
    treadmill_timestamps = treadmill_timestamps + abs(earliest_timestamp_in_behavior)
    frame_times = frame_times + abs(earliest_timestamp_in_behavior)
    movie_times = movie_times + abs(earliest_timestamp_in_behavior)
    trial_times["start_frame_time"] = trial_times["start_frame_time"] + abs(earliest_timestamp_in_behavior)
    trial_times["end_frame_time"] = trial_times["end_frame_time"] + abs(earliest_timestamp_in_behavior)



/tmp/ipykernel_3459/853074753.py:24: UserWarning: Writing behavior data to NWB with negative timestamps is not recommended,times are shifted to the earliest behavioral timestamp by 10.14944010375001 seconds.
  warn(


In [6]:
# Create the NWBFile
nwbfile = start_nwb(scan_key)
# Add eye position and pupil radius
add_eye_tracking(scan_key, nwbfile, timestamps=pupil_timestamps)
# Add the velocity of the treadmill
add_treadmill(scan_key, nwbfile, timestamps=treadmill_timestamps)
# Add trials
add_trials(scan_key, nwbfile, trial_times=trial_times)
# Add fluorescence traces, image masks and summary images to NWB
add_ophys(scan_key, nwbfile, timestamps=frame_times)


/usr/local/lib/python3.8/site-packages/pynwb/file.py:471: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [7]:
# pickle_directory = '/mnt/at-export01/17797-export/catalystNeuro/'
# # The file path to the movie timestamps pickle file (from v8 version)
# stimulus_movie_timestamps_file_path = Path(pickle_directory + 'v8_movie_timestamps.pkl')
# # The file path to the imaging timestamps pickle file (from v8 version)
# ophys_timestamps_file_path = Path(pickle_directory + 'ScanTimes.pkl')
# # The file path to the trials timestamps pickle file (from v8 version)
# trial_timestamps_file_path = Path(pickle_directory + 'Trial.pkl')

# # assert equivalency of fetching from nda.Trial v8 and previously provided Trial.pkl file (which was a frank fetch of the v8 Trial table)
# import pandas as pd
# for scan_key in scan_keys:
#     trial_df_1 = get_trial_times(scan_key=scan_key, file_path=trial_timestamps_file_path)
#     trial_df_2 = pd.DataFrame((nda.Trial & scan_key).fetch())
#     print(scan_key, trial_df_1.reset_index(drop=True).equals(trial_df_2))

# # assert equivalency of fetching from nda.ScanTimes v8 and previously provided ScanTimes.pkl file (which was a frank fetch of the v8 ScanTimes table)
# import numpy as np
# for scan_key in scan_keys:
#     frame_times_1 = get_frame_times(scan_key=scan_key, file_path=ophys_timestamps_file_path)
#     frame_times_2 = (nda.ScanTimes & scan_key).fetch1('frame_times')
#     print(scan_key,(np.array_equal(frame_times_1,frame_times_2)))

# # assert equivalency of generating from v8 and previously provided v8_movie_timestamps.pkl file (which was generated from v8 tables)
# import numpy as np
# for scan_key in scan_keys:
#     movie_times_1 = get_stimulus_times(scan_key=scan_key, file_path=stimulus_movie_timestamps_file_path)
#     movie_times_2,_ = resample_flips(scan_key)
#     print(scan_key,np.array_equal(movie_times_1,movie_times_2))



In [11]:
if verbose:
    print("Behavior, trials, and Fluorescence traces are added from datajoint.")

converter = MICrONSNWBConverter(source_data=source_data)
metadata = converter.get_metadata()

metadata["Behavior"]["Movies"][0].update(
    description="The visual stimulus is composed of natural movie clips ~60 fps.",
)

metadata["NWBFile"].update(
    session_start_time=nwbfile.session_start_time,
)

conversion_options = dict(
    Ophys=dict(stub_test=False),
    Video=dict(
        external_mode=False,
        timestamps=movie_times.tolist(),
    ),
)


Behavior, trials, and Fluorescence traces are added from datajoint.
Source data is valid!


In [12]:
metadata

{'NWBFile': {'session_description': 'Auto-generated by neuroconv',
  'identifier': 'd135f213-dba5-4c62-972b-729e5fae58a3',
  'session_start_time': datetime.datetime(2018, 3, 4, 0, 0, tzinfo=tzlocal())},
 'Ophys': {'TwoPhotonSeries': [{'name': 'TwoPhotonSeries1',
    'description': 'Calcium imaging data for field 1 at 6.3009 Hz and 8e-05 meters depth.',
    'imaging_plane': 'ImagingPlane1',
    'dimension': [248, 440],
    'field_of_view': [0.00062, 0.0011],
    'unit': 'n.a.'},
   {'name': 'TwoPhotonSeries2',
    'description': 'Calcium imaging data for field 2 at 6.3009 Hz and 8e-05 meters depth.',
    'imaging_plane': 'ImagingPlane2',
    'dimension': [248, 440],
    'field_of_view': [0.00062, 0.0011],
    'unit': 'n.a.'},
   {'name': 'TwoPhotonSeries3',
    'description': 'Calcium imaging data for field 3 at 6.3009 Hz and 0.00022 meters depth.',
    'imaging_plane': 'ImagingPlane3',
    'dimension': [248, 440],
    'field_of_view': [0.00062, 0.0011],
    'unit': 'n.a.'},
   {'name':

In [13]:
nwbfile

root pynwb.file.NWBFile at 0x140639783200416
Fields:
  acquisition: {
    EyeTracking <class 'pynwb.behavior.EyeTracking'>,
    PupilTracking <class 'pynwb.behavior.PupilTracking'>,
    treadmill_velocity <class 'pynwb.base.TimeSeries'>
  }
  devices: {
    Microscope <class 'pynwb.device.Device'>
  }
  experiment_description: The light microscopic images were acquired from a cubic millimeter volume that spanned portions of primary visual cortex and three higher visual cortical areas. The volume was imaged in vivo by two-photon random access mesoscope (2P-RAM) from postnatal days P75 to P81 in a male mouse expressing a genetically encoded calcium indicator in excitatory cells, while the mouse viewed natural movies and parametric stimuli. The calcium imaging data includes the single-cell responses of an estimated 75,000 pyramidal cells imaged over a volume of approximately 1200 x 1100 x 500 μm3 (anteroposterior x mediolateral x radial depth). The center of the volume was placed at the junction of primary visual cortex (VISp) and three higher visual areas, lateromedial area (VISlm), rostrolateral area (VISrl) and anterolateral area (VISal). During imaging, the animal was head-restrained, and the stimulus was presented to the left visual field. Treadmill rotation (single axis) and video of the animal's left eye were captured throughout the scan, yielding the locomotion velocity, eye movements, and pupil diameter data included here. The functional data were co-registered with electron microscopy (EM) data. The structural identifiers of the matched cells are added as plane segmentation columns extracted from the CAVE database. To access the latest revision see the notebook that is linked to this dandiset. The structural ids might not be present for all plane segmentations.
  file_create_date: [datetime.datetime(2023, 11, 18, 2, 41, 13, 870761, tzinfo=tzlocal())]
  identifier: b0345fc9-8be9-428d-baaf-a32861f41bb7
  imaging_planes: {
    ImagingPlane1 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane2 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane3 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane4 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane5 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane6 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane7 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane8 <class 'pynwb.ophys.ImagingPlane'>
  }
  intervals: {
    Clip <class 'pynwb.epoch.TimeIntervals'>,
    Monet2 <class 'pynwb.epoch.TimeIntervals'>,
    Trippy <class 'pynwb.epoch.TimeIntervals'>
  }
  processing: {
    ophys <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['https://doi.org/10.1101/2021.07.28.454025']
  session_description: Contains calcium imaging recorded from multiple cortical visual areas and behavioral measurements while a mouse viewed natural movies and parametric stimuli. The structural ids are added as plane segmentation columns from the CAVE database on 2023-11-18. To access the latest revision see the notebook that is linked to the dandiset. The structural ids might not be present for all plane segmentations.
  session_id: 4-scan-7
  session_start_time: 2018-03-04 00:00:00+00:00
  subject: subject pynwb.file.Subject at 0x140639783199456
Fields:
  age: P75D/P81D
  age__reference: birth
  sex: M
  species: Mus musculus
  subject_id: 17797

  timestamps_reference_time: 2018-03-04 00:00:00+00:00

In [15]:
try:
    converter.run_conversion(
        nwbfile=nwbfile,
        nwbfile_path=nwbfile_path,
        metadata=metadata,
        conversion_options=conversion_options,
    )
    if verbose:
        print("Conversion successful.")

    nwbfile_path = Path(nwbfile_path)
    # Run inspection for nwbfile
    results = list(inspect_nwb(nwbfile_path=nwbfile_path))
    report_path = nwbfile_path.parent / f"{nwbfile_path.stem}_report.txt"
    save_report(
        report_file_path=report_path,
        formatted_messages=format_messages(
            results,
            levels=["importance", "file_path"],
        ),
    )
    # # Upload nwbfile to DANDI
    # automatic_dandi_upload(
    #     dandiset_id="000402",
    #     nwb_folder_path=nwbfile_path.parent,
    #     cleanup=False,
    # )

    # if verbose:
    #     print("Cleaning up after successful upload to DANDI ...")
    # Path(ophys_file_path).unlink()
    # Path(stimulus_movie_file_path).unlink()

except Exception as e:
    warn(f"There was an error during conversion. The source files are not removed. The full traceback: {e}")


Metadata is valid!
conversion_options is valid!
TwoPhotonSeries data for plane 1 is added to nwbfile.
TwoPhotonSeries data for plane 2 is added to nwbfile.
TwoPhotonSeries data for plane 3 is added to nwbfile.
TwoPhotonSeries data for plane 4 is added to nwbfile.
TwoPhotonSeries data for plane 5 is added to nwbfile.
TwoPhotonSeries data for plane 6 is added to nwbfile.
TwoPhotonSeries data for plane 7 is added to nwbfile.
TwoPhotonSeries data for plane 8 is added to nwbfile.


/usr/local/lib/python3.8/site-packages/neuroconv/datainterfaces/behavior/video/videodatainterface.py:224: UserWarning: starting_times not provided, setting to 0.0
  warn("starting_times not provided, setting to 0.0")
Copying video data for stimulus_17797_4_7_v4.avi: 100%|██████████| 381054/381054 [1:28:32<00:00, 71.73it/s]    


NWB file saved at /mnt/at-export01/17797-export/DANDI_NWB/functional_scan_17797_4_7_v2/functional_scan_17797_4_7_v2.nwb!
Conversion successful.


/tmp/ipykernel_3459/4061038321.py:13: DeprecationWarning: The API function 'inspect_nwb' has been deprecated and will be removed in a future release! To remove ambiguity, please call either 'inspect_nwbfile' giving a path to the unopened file on a system, or 'inspect_nwbfile_object' passing an already open pynwb.NWBFile object.
  results = list(inspect_nwb(nwbfile_path=nwbfile_path))
